## Setup
#### Load the API key and relevant Python libaries.

In [1]:
import openai
from openai import OpenAI
import pprint as pp
import pandas as pd
import requests
import uuid
import os
from PIL import Image
from io import BytesIO
from datetime import datetime
import json
from HolidaysAPI import HolidayEvents
from dotenv import load_dotenv

load_dotenv()

True

### Prompts and Prompt functions

In [2]:
# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
openai.__version__

'1.42.0'

In [4]:
def query_gpt_chat(messages):
    model="gpt-4o"
    temperature=0.3
    max_tokens=3000
    response_format={ "type": "json_object" }
    
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
        response_format=response_format
    )
    
    gpt_response = response.choices[0].message.content
    return gpt_response

In [5]:
def generate_image(client, prompt):

    response = client.images.generate(
      model="dall-e-3",
      prompt=prompt,
      size="1024x1792",
      quality="standard",
      n=1,
    )

    image_url = response.data[0].url
    
    return image_url

In [6]:
country = "nigeria"
year = "2024"
start = "2024-12-01"
num_days = 35
place = "Kobar's Lounge"

In [7]:
delimiter = "#####"

## PROMPTS

In [8]:
output_format = "event:event name, persuasive_copy: persuasive_copy phrase"
persuasive_copy_system = f"""You are an experienced agent who makes bookings, \
as reservations for travellers or people on vacation, relaxation spots, etc for clients in {country}.\
User query will be delimited by {delimiter}
Follow these steps to answer the user's queries.
The user's query will contain an event in the format: event: name of event
The user's query will also contain a name of a hotel or place you are promoting for.
Step 1:{delimiter} You are to create engaging and culturally resonant call-to-action marketing phrases. \
aimed at the general public to spend the holiday at the hotel or place you are promoting\The phrase should:\
Highlight the unique appeal of the place in enhancing the holiday experience, \
emphasizing the emotional or experiential benefits.\
Step 2:{delimiter} Be written in authentic english,\
incorporating local slang and expressions to resonate with the target audience
Step 3:{delimiter} Ensure phrase is between 20 to 50 words, making them concise \
yet impactful for promotional materials or social media posts. and\
Include hashtags and appropriate emojis and smileys to mimic social media style and increase engagement, \
making each call to action visually appealing and shareable.
Step 4:{delimiter} Focus on creating a sense of urgency and excitement about the event, \
suggesting that spending the holiday in the place will not only enhance the holiday experience \
but also significantly be a memorable experience. \
Step 5:{delimiter} Return the output for each call to action as a json formatted like this {output_format}

Show only Step 5.
"""

In [9]:
holiday_image_gen = f"""The user's query will contain an event.
Create a vibrant and engaging promotional poster for that event. \
The poster should capture the essence of the event, \
incorporating relevant symbols, colors, and themes that resonate with the occasion. \
Make sure to include festive elements and dynamic visuals that highlight the celebration. \
Include the name of the event on the poster.
"""

## CLEAN JSON

In [10]:
def clean_gpt_json(gpt_json):
    clean_json = f"{persuasive_copy}"
    if "\n" in persuasive_copy:
        print("cleaning new lines")
        #clean_json = clean_json.replace("\n\n", ',')
        new_lines = ["\n\n", "\n"]
        for xter in new_lines:
            clean_json = clean_json.replace(xter,"")
        clean_json = f'[{clean_json}]'
    clean_json = json.loads(clean_json)
    return clean_json

#### fetch events

In [11]:
def get_all_events_table(country,year,start=False, num_days=30):
    events_api = HolidayEvents(country, year)
    calendar_events = events_api.filter_events(start, num_days)
    return calendar_events

In [12]:
df = get_all_events_table(country,year,start,num_days)
df

INFO:root:========== fetching holiday events ==========
/Users/taiwoadelakin/Documents/Doc/Projects/digital-content-creation-for-holiday-promotionss-using-openai-gpt4-and-dalle3-models/HolidaysAPI.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_df['date'] = event_df['date'].dt.strftime("%Y-%m-%d %H:%M:%S")


,date,day,event,category,eventday,location,month,static_or_dynamic,source,week
0,2024-12-21 00:00:00,saturday,december solstice,season,2024-12-21,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-51
1,2024-12-22 00:00:00,sunday,sambisa memorial day,local holiday,2024-12-22,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-51
2,2024-12-24 00:00:00,tuesday,christmas eve,"observance, christian",2024-12-24,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-52
3,2024-12-25 00:00:00,wednesday,christmas day,public holiday,2024-12-25,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-52
4,2024-12-26 00:00:00,thursday,boxing day,public holiday,2024-12-26,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-52
5,2024-12-31 00:00:00,tuesday,new years eve,observance,2024-12-31,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-53


In [13]:
len(df)

6

#### Loading Events from CSV

In [14]:
#USE THIS CELL ONLY WHEN READING FROM EXISTING CSV
# Read the input CSV file
# input_file = '/Users/udhtaztaiwo/Documents/september_events.csv'

# try:
#     df = pd.read_csv(input_file)
#     df.drop('Unnamed: 0', axis=1, inplace=True)
# except FileNotFoundError:
#     print(f"File {input_file} not found. Please make sure the file exists.")
#     exit()

In [15]:
# # Convert the 'Date' column to datetime
# df['date'] = pd.to_datetime(df['date'],dayfirst=True)

# # Get unique weeks from the CSV file
# df['week'] = df['date'].dt.strftime('%Y-%U')
# unique_weeks = df['week'].unique()
# df.head(10)


### Call to action

In [16]:
persuasive_copy_list = []

for event in df['event']:
    print("="*5, f"call to action for {event}", "="*5)
    event = f"event: {event}"
    recommended_place = place

    messages =  [{'role':'system', 'content': persuasive_copy_system},
             {'role':'user', 'content': f"{delimiter}{event}{delimiter}{recommended_place}{delimiter}"}]
    persuasive_copy = query_gpt_chat(messages)
    persuasive_copy_json = clean_gpt_json(persuasive_copy)
    print(persuasive_copy_json)
    print("\n")
            
    if isinstance(persuasive_copy_json, dict):
        persuasive_copy_list.append(persuasive_copy_json)
    else:
        persuasive_copy_list.extend(persuasive_copy_json)


===== call to action for december solstice =====


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


cleaning new lines
[{'event': 'december solstice', 'persuasive_copy': "Celebrate the December Solstice at Kobar's Lounge! 🌟 Enjoy vibes, good food, and unforgettable moments. Don't miss out on the ultimate holiday experience! #KobarsLounge #DecemberSolstice #UnwindAndShine 🌞🍹"}]


===== call to action for sambisa memorial day =====


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


cleaning new lines
[{'event': 'sambisa memorial day', 'persuasive_copy': "Celebrate Sambisa Memorial Day at Kobar's Lounge! Enjoy top-notch vibes, delicious local dishes, and unforgettable moments. Don't miss out, book now! 🌟🍲 #SambisaMemorialDay #KobarsLounge #NaijaVibes #UnforgettableExperience"}]


===== call to action for christmas eve =====


...

cleaning new lines
[{'event': 'christmas day', 'persuasive_copy': "Celebrate Christmas Day in style at Kobar's Lounge! 🎄✨ Enjoy the festive vibes, delicious treats, and unforgettable moments with loved ones. Don't miss out, book now! #ChristmasAtKobars #FestiveFun #UnforgettableMoments 🎅🏽🎁"}]


===== call to action for boxing day =====


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


cleaning new lines
[{'event': 'boxing day', 'persuasive_copy': "This Boxing Day, come chill at Kobar's Lounge! Enjoy vibes, good food, and unforgettable moments. Don't miss out on the ultimate holiday experience! 🎉🍹 #BoxingDayChill #KobarsLounge #UnwindAndRelax"}]


===== call to action for new years eve =====


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


cleaning new lines
[{'event': 'new years eve', 'persuasive_copy': "Celebrate New Year's Eve in grand style at Kobar's Lounge! 🎉✨ Enjoy the vibes, the beats, and the unforgettable moments. Don't miss out, book now and make it a night to remember! #NewYearsEve #KobarsLounge #TurnUpNaija"}]




In [17]:
persuasive_copy_df = pd.DataFrame(persuasive_copy_list)

# merge both dataframes 
copy_df = df.merge(persuasive_copy_df, on = 'event', how = 'left')
copy_df.head()

,date,day,event,category,eventday,location,month,static_or_dynamic,source,week,persuasive_copy
0,2024-12-21 00:00:00,saturday,december solstice,season,2024-12-21,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-51,Celebrate the December Solstice at Kobar's Lou...
1,2024-12-22 00:00:00,sunday,sambisa memorial day,local holiday,2024-12-22,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-51,Celebrate Sambisa Memorial Day at Kobar's Loun...
2,2024-12-24 00:00:00,tuesday,christmas eve,"observance, christian",2024-12-24,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-52,Celebrate Christmas Eve in style at Kobar's Lo...
3,2024-12-25 00:00:00,wednesday,christmas day,public holiday,2024-12-25,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-52,Celebrate Christmas Day in style at Kobar's Lo...
4,2024-12-26 00:00:00,thursday,boxing day,public holiday,2024-12-26,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-52,"This Boxing Day, come chill at Kobar's Lounge!..."


### Generate image url

In [18]:
event_image_list = []


for event in copy_df['event']:
    print("="*5, f"generating image for {event}", "="*5)
    event_name = f"I want to generate an image for event: {event}."
    image_url = generate_image(client, f"{event_name} {holiday_image_gen}")
    event_image_json = {"event": event, "image": image_url}
    
    if isinstance(event_image_json, dict):
        event_image_list.append(event_image_json)
    else:
        event_image_list.extend(event_image_json)    
        

===== generating image for december solstice =====


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/images/generations "HTTP/1.1 200 OK"


===== generating image for sambisa memorial day =====


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/images/generations "HTTP/1.1 200 OK"


===== generating image for christmas eve =====


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/images/generations "HTTP/1.1 200 OK"


===== generating image for boxing day =====


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/images/generations "HTTP/1.1 200 OK"


===== generating image for new years eve =====


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/images/generations "HTTP/1.1 200 OK"


In [19]:
event_image_list

[{'event': 'december solstice',
  'image': 'https://oaidalleapiprodscus.blob.core.windows.net/private/org-T85ZkjXfXsQLdxcCNnnrUda5/user-jZARJYRxwfvb5DM8eBiVW6Y4/img-HsoV3NMgEIJVWUEAaWQH4DPy.png?st=2024-09-19T09%3A17%3A24Z&se=2024-09-19T11%3A17%3A24Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-09-19T02%3A05%3A35Z&ske=2024-09-20T02%3A05%3A35Z&sks=b&skv=2024-08-04&sig=M4gTppqSlyxkp5vHnicqI0GE/5hLVvDLp9N5j%2B/x6QI%3D'},
 {'event': 'sambisa memorial day',
  'image': 'https://oaidalleapiprodscus.blob.core.windows.net/private/org-T85ZkjXfXsQLdxcCNnnrUda5/user-jZARJYRxwfvb5DM8eBiVW6Y4/img-D94JOHGuQM2pKxXnpaBK3dow.png?st=2024-09-19T09%3A17%3A46Z&se=2024-09-19T11%3A17%3A46Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-09-19T02%3A01%3A21Z&ske=2024-09-20T02%3A01%3A21Z&sks=b&skv=2024-08-04&sig=nSXdI

In [20]:
event_image_df = pd.DataFrame(event_image_list)

# merge both dataframes 
final_df = copy_df.merge(event_image_df, on = 'event', how = 'left')
final_df.head()

,date,day,event,category,eventday,location,month,static_or_dynamic,source,week,persuasive_copy,image
0,2024-12-21 00:00:00,saturday,december solstice,season,2024-12-21,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-51,Celebrate the December Solstice at Kobar's Lou...,https://oaidalleapiprodscus.blob.core.windows....
1,2024-12-22 00:00:00,sunday,sambisa memorial day,local holiday,2024-12-22,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-51,Celebrate Sambisa Memorial Day at Kobar's Loun...,https://oaidalleapiprodscus.blob.core.windows....
2,2024-12-24 00:00:00,tuesday,christmas eve,"observance, christian",2024-12-24,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-52,Celebrate Christmas Eve in style at Kobar's Lo...,https://oaidalleapiprodscus.blob.core.windows....
3,2024-12-25 00:00:00,wednesday,christmas day,public holiday,2024-12-25,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-52,Celebrate Christmas Day in style at Kobar's Lo...,https://oaidalleapiprodscus.blob.core.windows....
4,2024-12-26 00:00:00,thursday,boxing day,public holiday,2024-12-26,nigeria,December,static,www.timeanddate.com/holidays/nigeria,2024-52,"This Boxing Day, come chill at Kobar's Lounge!...",https://oaidalleapiprodscus.blob.core.windows....


In [21]:
output_file = 'holidays.csv'
final_df.to_csv(output_file, index=False)